<a href="https://colab.research.google.com/github/merwan32/nlp-getting-started/blob/main/solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"bekkarmerwan","key":"27cc0cceb4716c6ed5ff1e1214321d12"}'}

In [3]:
# Move the uploaded JSON file to the Kaggle directory
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

# Set permissions
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
# Use the Kaggle API to download the competition data
!kaggle competitions download -c nlp-getting-started


  0% 0.00/593k [00:00<?, ?B/s]
100% 593k/593k [00:00<00:00, 159MB/s]


In [5]:
# Unzip the downloaded data
!unzip nlp-getting-started.zip -d nlp-getting-started

# List the files in the directory
!ls nlp-getting-started


Archive:  nlp-getting-started.zip
  inflating: nlp-getting-started/sample_submission.csv  
  inflating: nlp-getting-started/test.csv  
  inflating: nlp-getting-started/train.csv  
sample_submission.csv  test.csv  train.csv


In [6]:
!ls

nlp-getting-started  nlp-getting-started.zip  sample_data


In [8]:
import pandas as pd

# Load training data
train_data = pd.read_csv('/content/nlp-getting-started/train.csv')
test_data = pd.read_csv('/content/nlp-getting-started/test.csv')


In [10]:
train_data['keyword'].fillna('', inplace=True)
train_data['location'].fillna('', inplace=True)

In [12]:
import nltk
import re
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:
nltk.download('wordnet')
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

[nltk_data] Downloading package wordnet to /root/nltk_data...


unzip:  cannot find or open /usr/share/nltk_data/corpora/wordnet.zip, /usr/share/nltk_data/corpora/wordnet.zip.zip or /usr/share/nltk_data/corpora/wordnet.zip.ZIP.


In [15]:
en_stopwords = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    # convert to lowercase
    text = text.lower()

    # filter text 1.mentions, 2.urls, 3.htmltags, 4.punctuation, 5.emojis,
    url_pattern = r'http[s]?://\S+'
    html_tags = r'<.*?>'
    mentions = r'@\w+'
    punctuation = r'[^\w\s]'
    text = re.sub(mentions,'',text)
    text = re.sub(url_pattern,'',text)
    text = re.sub(html_tags,'',text)
    text = re.sub(punctuation,'',text)

    # tokenize using word_tokenize nltk & filter 1.stopword, 3.numbers,
    text = word_tokenize(text)
    text = [word for word in text if word not in en_stopwords]
    text = [word for word in text if not word.isdigit()]

    # stemmeing words
    text = [lemmatizer.lemmatize(word) for word in text]

    return ' '.join(text)



In [19]:
train_data['text'] = train_data['text'].apply(preprocess_text)

In [20]:
test_data['text'] = test_data['text'].apply(preprocess_text)

In [22]:
from sklearn.model_selection import train_test_split
train_data['tweet'] = train_data['text'] + ' ' + train_data['keyword'] + ' ' + train_data['location']
X = train_data['tweet']
y = train_data['target']

In [24]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

In [26]:
from sklearn.naive_bayes import MultinomialNB

# Create a Multinomial Naive Bayes model
nb_model = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),
    ('classifier', MultinomialNB())
])

nb_model.fit(X_train, y_train)

val_predictions = nb_model.predict(X_val)
accuracy = accuracy_score(y_val, val_predictions)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.79


In [28]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.0 MB/s eta 0:00:00


In [29]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Tokenize the training data
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, return_tensors='pt')

# Tokenize the validation data
val_encodings = tokenizer(list(X_val), truncation=True, padding=True, return_tensors='pt')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
import torch

# Convert labels to tensors
y_train_tensor = torch.tensor(y_train.values)
y_val_tensor = torch.tensor(y_val.values)

# Create data loaders
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [31]:
from transformers import BertForSequenceClassification, AdamW
import torch
import torch.nn as nn
from tqdm import tqdm

# Define the loss function
loss_fn = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Number of training epochs
num_epochs = 3

# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}'):
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        batch_loss = loss_fn(logits, labels)
        batch_loss.backward()
        optimizer.step()
        train_loss += batch_loss.item()

    # Calculate and print average training loss for the epoch
    avg_train_loss = train_loss / len(train_loader)
    print(f'Epoch {epoch + 1}, Average Training Loss: {avg_train_loss:.4f}')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1: 100%|██████████| 191/191 [41:41<00:00, 13.10s/it]


Epoch 1, Average Training Loss: 0.4928


Epoch 2: 100%|██████████| 191/191 [42:22<00:00, 13.31s/it]


Epoch 2, Average Training Loss: 0.3820


Epoch 3: 100%|██████████| 191/191 [43:20<00:00, 13.61s/it]

Epoch 3, Average Training Loss: 0.3259


In [32]:
test_encodings = tokenizer(list(test_data['text']), truncation=True, padding=True, return_tensors='pt')

In [33]:
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'])
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)  # No need to shuffle the test set


In [34]:
model.eval()  # Set the model to evaluation mode
predictions = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Inference on Test Data'):
        input_ids, attention_mask = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)  # Calculate class probabilities
        predictions.extend(probabilities[:, 1].tolist())  # Append the probability of class 1 (disaster) to predictions

# Convert probabilities to binary predictions (0 or 1) based on a threshold (e.g., 0.5)
threshold = 0.5
binary_predictions = [1 if p >= threshold else 0 for p in predictions]


Inference on Test Data: 100%|██████████| 102/102 [06:57<00:00,  4.09s/it]


In [35]:
submission_df = pd.DataFrame({'id': test_data['id'], 'target': binary_predictions})


In [36]:
submission_df.to_csv('submission.csv', index=False)
